In [1]:
import pandas as pd
import numpy as np
import os
import math
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler, StandardScaler
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
dir_path = './' #local
# dir_path = './drive/MyDrive/' #colab

In [2]:
#uncomment in colab
# from google.colab import drive
# drive.mount('/content/drive')
# pd.read_csv(dir_path + 'DataCollection' + '/' + '1' + '/' + '0.txt', sep=' ', index_col=False )
# test = np.genfromtxt(dir_path + 'DataCollection' + '/' + '1' + '/' + '0.txt', delimiter=' ',dtype='float64')
df = pd.read_csv(dir_path + 'DataCollection' + '/' + '1' + '/' + '0.txt', header=None, sep=' ')
df = df.drop(columns=[31])
df

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,152.028,174.270,114.872,135.012,115.821,169.357,108.251,86.5737,86.4014,60.6628,...,19.15310,37.64440,296.277,5.27758,5.69833,303.563,-4.53744,7.62196,12.33870,35.1608
1,106.969,174.270,138.093,122.560,112.017,151.941,139.409,67.5126,93.4567,141.7770,...,0.97681,54.97550,295.547,-61.05800,8.98806,317.394,-4.13179,85.07790,5.64757,43.2827
2,115.931,174.270,114.917,134.286,175.119,127.521,177.996,58.0159,69.8662,178.0840,...,-2.39748,82.31940,336.737,-44.98200,13.94950,337.090,-7.44972,63.64330,21.32290,29.9727
3,129.774,90.000,151.379,164.207,147.675,133.193,153.299,53.3455,64.2682,55.6463,...,6.15326,45.33460,348.796,1.37925,19.70390,370.834,-9.76106,19.74900,17.69820,71.4496
4,168.765,174.270,156.177,141.628,146.640,128.245,177.981,80.4555,66.8833,71.7721,...,7.91760,45.88860,362.047,13.63630,27.27890,388.642,-8.96836,15.10440,19.16930,97.3733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,149.508,178.660,144.652,141.589,141.211,140.317,173.599,76.2446,75.0304,72.7944,...,-30.11830,2.87239,522.783,-37.79160,36.79610,518.710,-48.87630,12.26400,7.59416,117.8630
196,149.347,178.758,145.400,142.273,141.907,139.853,173.076,75.6861,74.6439,72.5005,...,-29.44780,3.12718,520.981,-36.96540,36.67920,517.278,-48.70080,12.25370,7.67346,118.6160
197,148.335,178.866,146.606,143.288,142.679,139.443,172.818,75.3177,74.4949,72.3500,...,-28.72460,3.23291,518.414,-36.05660,36.29120,515.412,-48.58660,12.25620,7.78125,119.4990
198,146.453,178.656,149.390,144.159,143.569,137.445,173.159,74.5098,73.8029,71.7551,...,-28.28420,3.47881,515.864,-35.43990,35.86260,513.897,-48.75530,12.21440,7.67485,119.8710


In [3]:
files = os.listdir(dir_path + 'DataCollection')
files

['1', '2', '3', '4', '5', '6']

In [4]:
x_train = [];
y_train = [];

x_test = [];
y_test = [];
for i in files:
    samples = os.listdir(dir_path + 'DataCollection' + '/' + i)
    num_tests = int(len(samples)/5);
    shuffle(samples, random_state = 0)
    for k in range(0, num_tests):
#         df = np.genfromtxt(dir_path + 'DataCollection' + '/' + i + '/' + samples[k], delimiter=' ',dtype='float64')
        df = pd.read_csv(dir_path + 'DataCollection' + '/' + i + '/' + samples[k], header=None, sep=' ')
        df = df.drop(columns=[31])
        x_test.append(df.to_numpy())
        y_test.append(int(i)-1);
    
    for k in range(num_tests, len(samples)):
        df = pd.read_csv(dir_path + 'DataCollection' + '/' + i + '/' + samples[k], header=None, sep=' ')
        df = df.drop(columns=[31])
        x_train.append(df.to_numpy())
        y_train.append(int(i)-1);
    
    print(len(samples), ' ', num_tests)
    
x_train = np.array(x_train)
y_train = np.array(y_train);

x_test = np.array(x_test)
y_test = np.array(y_test)

301   60
256   51
151   30
205   41
211   42
151   30


In [5]:
print("x_train.shape: ", x_train.shape)
print("y_train.shiape: ", y_train.shape)
print("x_test.shape: ", x_test.shape)
print("y_test.shape: ", y_test.shape)

print(x_train)

x_train.shape:  (1021, 200, 31)
y_train.shiape:  (1021,)
x_test.shape:  (254, 200, 31)
y_test.shape:  (254,)
[[[146.102   174.27    146.022   ...  12.747     8.94849 116.923  ]
  [138.963   176.016   145.252   ...  12.831     9.10288 116.791  ]
  [133.62    177.874   145.35    ...  13.1807    9.07881 117.362  ]
  ...
  [154.893   172.515   141.491   ...  12.4594    7.48152 112.73   ]
  [154.995   173.132   141.052   ...  12.4302    7.43961 112.92   ]
  [155.21    173.614   140.785   ...  12.4084    7.43464 112.993  ]]

 [[155.597   173.91    140.623   ...  12.3902    7.44884 113.069  ]
  [155.597   172.187   141.973   ...  12.5572    7.3875  113.661  ]
  [155.573   172.146   141.634   ...  12.532     7.41295 113.582  ]
  ...
  [159.124   172.102   143.974   ...  11.8476    8.21762 111.365  ]
  [159.203   172.155   143.993   ...  11.863     8.25759 111.437  ]
  [159.278   172.184   143.972   ...  11.8687    8.29868 111.479  ]]

 [[159.337   172.231   143.909   ...  11.8769    8.32961 11

In [6]:
def scale_data(data, min_max_scaler):
    for i in range(len(data)):
        data[i] = min_max_scaler.transform(data[i])
    return data

In [7]:
min_max_scaler = MinMaxScaler(feature_range=(0,1))

num_instances, num_time_steps, num_features = x_train.shape
x_train = np.reshape(x_train, newshape=(-1, num_features))
x_train = min_max_scaler.fit_transform(x_train)
x_train = np.reshape(x_train, newshape=(num_instances, num_time_steps, num_features))

x_train, y_train = shuffle(x_train, y_train, random_state=0)

num_instances, num_time_steps, num_features = x_test.shape
x_test = np.reshape(x_test, newshape=(-1, num_features))
x_test = min_max_scaler.transform(x_test)
x_test = np.reshape(x_test, newshape=(num_instances, num_time_steps, num_features))

x_test, y_test = shuffle(x_test, y_test, random_state=0)


In [8]:
# x_train = np.delete(x_train, [0,1,2,3,4,5,6,7,8], 0)
# y_train = np.delete(y_train, [0,1,2,3,4,5,6,7,8], 0)
print("x_train.shape: ", x_train.shape)
print("y_train.shiape: ", y_train.shape)
print("x_test.shape: ", x_test.shape)
print("y_test.shape: ", y_test.shape)
# print(y_train)
# print("\n")
# print(y_test)

# y_train = y_train.astype('int')

# print(y_train)


x_train.shape:  (1021, 200, 31)
y_train.shiape:  (1021,)
x_test.shape:  (254, 200, 31)
y_test.shape:  (254,)


In [11]:
model = Sequential()
model.add(LSTM(units=200, input_shape=x_train.shape[1:], return_sequences=True,dtype='float64'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(LSTM(200, return_sequences=True ,dtype='float64'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(LSTM(200, return_sequences=True ,dtype='float64'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(LSTM(200,dtype='float64'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

# model.add(Dense(30, activation='softmax'))

model.add(Dense(6, activation='softmax',dtype='float64'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 200, 200)          185600    
_________________________________________________________________
dropout_4 (Dropout)          (None, 200, 200)          0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 200, 200)          320800    
_________________________________________________________________
dropout_5 (Dropout)          (None, 200, 200)          0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 200, 200)          320800    
_________________________________________________________________
dropout_6 (Dropout)          (None, 200, 200)          0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 200)              

In [ ]:
opt = tf.keras.optimizers.Adam(lr=0.0001)

checkpoint_filepath = 'Checkpoints/'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'],
)


gestures = model.fit(x = x_train,
            y = y_train,
            epochs=1,
            validation_split=0.1, #split 10% of the trainning set for the validation set,
            batch_size=80,
            callbacks=[model_checkpoint_callback],
            shuffle=True
         )

 1/12 [=>............................] - ETA: 1:11 - loss: 1.7926 - accuracy: 0.1625

In [15]:
# model.save(dir_path + 'Models/gestures.h5')

In [16]:
print("Evaluate on test data")
results = model.evaluate(x_test, y_test, batch_size=80)
print("test loss, test acc:", results)
model.save(dir_path + 'Models/gestures_custom', save_format='tf')

Evaluate on test data
5/5 [==============================] - 0s 37ms/step - loss: 0.6697 - accuracy: 0.8535
test loss, test acc: [0.6696683764457703, 0.8534704446792603]


INFO:tensorflow:Assets written to: ./drive/MyDrive/Models/gestures/assets


INFO:tensorflow:Assets written to: ./drive/MyDrive/Models/gestures/assets


In [12]:
# new_model = load_model(dir_path + 'Models/gestures.h5')
# print("Evaluate on test data")
# results = new_model.evaluate(x_test, y_test, batch_size=80)
# print("test loss, test acc:", results)

Evaluate on test data


ValueError: in user code:

    C:\Users\tranv\anaconda3\envs\baka\lib\site-packages\tensorflow\python\keras\engine\training.py:1478 predict_function  *
        return step_function(self, iterator)
    C:\Users\tranv\anaconda3\envs\baka\lib\site-packages\tensorflow\python\keras\engine\training.py:1468 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\tranv\anaconda3\envs\baka\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\tranv\anaconda3\envs\baka\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\tranv\anaconda3\envs\baka\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\tranv\anaconda3\envs\baka\lib\site-packag